<img src="img/title-card.png" width="720" title="ThoughtWorks presents Pythonic Objects by Luciano Ramalho">

# Attribute basics

## Smalltalk class declaration

<img src="img/finhist-browser.png" width="720" title="Smallalk-80 class browser">

**Figure 17.17 from Smalltalk-80, the language**
<br>Class browser showing definition of a ``FinancialHistory`` class with three instance variables: ``cashOnHand``, ``incomes``, and ``expenditures``.

## Official Java tutorial

The next two figures are from the **Java Tutorial (Sun/Oracle)**, section [What is an object?](https://docs.oracle.com/javase/tutorial/java/concepts/object.html).

<img src="img/concepts-object.gif" title="An object">

An object is depicted as fields surrounded by methods. 

<img src="img/concepts-bicycleObject.gif" title="A bicycle object">

An object representing a bicyle has methods such as *Change gear* and *Brake*, and fields such as *speed* and *cadence*.

Code from section [[What is a class?](https://docs.oracle.com/javase/tutorial/java/concepts/class.html) from the **Java Tutorial**.

```java
class Bicycle {

    int cadence = 0;
    int speed = 0;
    int gear = 1;

    void changeCadence(int newValue) {
         cadence = newValue;
    }
    
    //...
}    
```


## Python before ``typing``

* No way to declare variables (except function arguments).
* First assignment is the "declaration".
* ORMs use *descriptors* to declare fields (eg. Django, SQLAlchemy)

### Descriptor examples

From [Django Models](https://docs.djangoproject.com/en/2.2/topics/db/models/):

```python
from django.db import models

class Musician(models.Model):
    first_name = models.CharField(max_length=50)
    last_name = models.CharField(max_length=50)
    instrument = models.CharField(max_length=100)

class Album(models.Model):
    artist = models.ForeignKey(Musician, on_delete=models.CASCADE)
    name = models.CharField(max_length=100)
    release_date = models.DateField()
    num_stars = models.IntegerField()
```

----

From [SQL Alquemy Object Relational Tutorial](https://docs.sqlalchemy.org/en/13/orm/tutorial.html):

```python
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String

Base = declarative_base()

class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)

    def __repr__(self):
       return "<User(name='%s', fullname='%s', nickname='%s')>" % (
                            self.name, self.fullname, self.nickname)
```

In [4]:
import sys
print(sys.version)

3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


## A simplistic class

In [5]:
class Coordinate:
    '''Coordinate on Earth'''

In [6]:
cle = Coordinate()
cle.lat = 41.411667
cle.long = -81.849722
cle

## First method: ``__repr__``

In [27]:
import geohash

class Coordinate:
    '''Coordinate on Earth'''
        
    def __repr__(self):
        return f'Coordinate({self.lat}, {self.long})'   

In [32]:
cle = Coordinate()
try:
    print(cle)
except AttributeError as exc:
    print(exc)

'Coordinate' object has no attribute 'lat'


> **Quick fix**: add class attributes to provide defaults.

In [18]:
import geohash

class Coordinate:
    '''Coordinate on Earth'''
    
    def __init__(self, lat=0.0, long=0.0):
        self.lat = lat
        self.long = long
    
    def __repr__(self):
        return f'Coordinate({self.lat}, {self.long})'

In [14]:
cle = Coordinate(41.411667, -81.849722)
cle

Coordinate(41.411667, -81.849722)

In [16]:
import geohash

class Coordinate:
    '''Coordinate on Earth'''
    
    reference_system = 'WGS84'
    
    def __init__(self, lat, long):
        self.lat = lat
        self.long = long
    
    def __repr__(self):
        return f'Coordinate({self.lat}, {self.long})'
    
    def __str__(self):
        ns = 'NS'[self.lat < 0]
        we = 'WE'[self.long < 0]
        return f'{abs(self.lat):.1f}°{ns}, {abs(self.long):.1f}°{we}'
    
    def geohash(self):
        return geohash.encode(self.lat, self.long)
    

In [17]:
cle = Coordinate(41.411667, -81.849722)
cle.geohash()

'dpmg96yb7npz'

In [14]:
from dataclasses import InitVar
from typing import ClassVar

In [10]:
default_flavor = 'Cheese'

class Pizza:
    
    def __init__(self, flavor1=default_flavor, flavor2=None):
        self.flavor1 = flavor1
        self.flavor2 = flavor2

In [13]:
InitVar?

----

## dataclass options

```
@dataclasses.dataclass(*, 
    init=True, repr=True, eq=True, order=False, unsafe_hash=False, frozen=False)
```

<table>
<tr><th>option</th><th>default</th><th style="text-align: left;">meaning</th></tr>
<tr><td>init</td><td>True</td>
    <td style="text-align: left;">generate <code>__init__</code>¹</td></tr>
<tr><td>repr</td><td>True</td>
    <td style="text-align: left;">generate <code>__repr__</code>¹</td></tr>
<tr><td>eq</td><td>True</td>
    <td style="text-align: left;">generate <code>__eq__</code>¹</td></tr>
<tr><td>order</td><td>False</td>
    <td style="text-align: left;">generate <code>__lt__</code>, <code>__le__</code>, <code>__gt__</code>, <code>__ge__</code>²</td></tr>
<tr><td>unsafe_hash</td><td>False</td>
    <td style="text-align: left;">generate <code>__hash__</code>³</td></tr>
<tr><td>frozen</td><td>False</td>
    <td style="text-align: left;">make instances "immutable" ⁴</td></tr>
</table>

**Notes**

¹ Ignored if the special method is implemented by user.<br>
² Raises exceptions if ``eq=False`` or any of the listed special methods are implemented by user.<br>
³ Complex semantics and several caveats — see: [dataclass documentation](https://docs.python.org/3/library/dataclasses.html#dataclasses.dataclass).<br>
⁴ Not really immutable — imutability is emulated generating ``__setattr__`` and ``__delattr__`` which raise ``dataclass.FrozenInstanceError`` (a subclass of ``AttributeError``).

In [17]:
import dataclasses
dataclasses.FrozenInstanceError.__mro__

(dataclasses.FrozenInstanceError,
 AttributeError,
 Exception,
 BaseException,
 object)

<img src="img/thoughtworks.png" width="300" title="ThoughtWorks, Inc. logo">
